In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 100% !important; }</style>"))

/tmp/ipykernel_202031/3274186680.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
import torch

torch.cuda.is_available()
#!/usr/bin/env python3

import sys,os
from elftools.elf.elffile import ELFFile
from elftools.elf.segments import Segment
from capstone import *
from capstone.x86 import *

import os
import json 

import torch
from torch.utils.data import DataLoader, TensorDataset
import sys,os
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'


In [3]:

MAX_TOKEN_LEN = 1024
BATCH_SIZE = 40
epochs = 10

EXPERIMENT_NAME = 'cusTokenizer_UNI_35k_ASIS'
new_vocab_size = 35000
# disassembly_decimal disassembly_all_number_to_words disassembly_decimal 
data_key = "disassembly_decimal"


from transformers import BertTokenizer, BertForNextSentencePrediction,BertForPreTraining,BertConfig,AutoModelForMaskedLM,get_linear_schedule_with_warmup ,BertForSequenceClassification
import torch
from transformers import PreTrainedTokenizerFast ,AutoModelForPreTraining ,BertGenerationEncoder, BertGenerationDecoder,EncoderDecoderModel
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer = PreTrainedTokenizerFast.from_pretrained("./../models/" + EXPERIMENT_NAME)
print(tokenizer.pad_token) 

[PAD]


In [4]:
def generate_param_string(param_list):


    if len(param_list)==0:
        return 'void'
    else:
        return " ".join(param_list)
def unique_items(list_of_lists):
    # Flatten the list of lists
    print(list_of_lists[3].split())
    flattened_list = [item for sublist in list_of_lists for item in sublist.split()]
    # Convert the flattened list to a set to remove duplicates
    unique_elements = set(flattened_list)
    return list(unique_elements)


# DATASET GENERATION

In [5]:



DATA_PATH = '/home/raisul/ANALYSED_DATA/tokenization_data_single_functions'

TRAIN_DATA_PATH  ='/home/raisul/ANALYSED_DATA/tokenization_data_single_functions/train/'

TEST_DATA_PATH   = '/home/raisul/ANALYSED_DATA/tokenization_data_single_functions/test/'




train_json_files = [os.path.join(TRAIN_DATA_PATH, f) for f in os.listdir(TRAIN_DATA_PATH) ]

test_json_files = [os.path.join(TEST_DATA_PATH, f) for f in os.listdir(TEST_DATA_PATH) ]






def read_corpus(json_files):

    all = []
    all_params = []
    all_returns = []

    for k, j_file in enumerate(json_files):
        if k>1000:
            break
        try:

            with open(j_file, 'r') as file:
                data = json.load(file)

                funct = data[data_key]['input']
                
                all.append(funct)

                all_params.append( generate_param_string( data['signature']['paramTypes'] ) )
                all_returns.append(  data['signature']['returnType']  )
        except Exception as e: 
            print(e)
    return all , all_params , all_returns
    


train_text , train_params , train_returns = read_corpus(train_json_files)



test_text , test_params , test_returns  = read_corpus(test_json_files)


        
# text = text[0:5000]
print("Functions Count: ",len(train_text), '\n')
example = train_text[10]
text = train_text + test_text


unique_types = unique_items(train_params)
print(len(unique_types), '\n',unique_types)



Functions Count:  1001 

['int', '*struct']
21 
 ['*long', '*void', 'uint', '*uchar', 'void', 'ulonglong', 'char', 'float', '*longlong', 'longlong', '*struct', 'long', '*float', 'short', 'double', 'ulong', 'None', '*int', '*char', '*double', 'int']


In [6]:
def tokenize(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=1024, return_tensors="pt")

source_encodings = tokenize(train_text)
target_encodings = tokenize(train_params)

# Creating the pipeline

In [7]:



# encoder = BertGenerationDecoder.from_pretrained("./../models/"+EXPERIMENT_NAME+"_model.ckpt" ,bos_token_id=101, eos_token_id=102 )
# # decoder = BertGenerationDecoder.from_pretrained("./../models/"+EXPERIMENT_NAME+"_model.ckpt" ,  add_cross_attention=True, is_decoder=True, bos_token_id=101, eos_token_id=102 )

# decoder = BertGenerationDecoder.from_pretrained(
#     "google-bert/bert-large-uncased", add_cross_attention=True, is_decoder=True, bos_token_id=101, eos_token_id=102
# )

# bert2bert = EncoderDecoderModel(encoder=encoder, decoder=decoder)




# config = BertConfig.from_pretrained("microsoft/MiniLM-L12-H384-uncased")
# # Change max_position_embeddings to 1024 in the config
# config.max_position_embeddings = 1024
# # Load the model with the modified config
# model = BertForSequenceClassification(config)
# # Access and modify the positional embeddings
# # The model's `bert` attribute holds the BERT layers
# model_bert = model.bert  # This is the BERT model itself (the backbone)
# # Resize the position embeddings to accommodate the new max length (1024 tokens)
# model_bert.embeddings.position_embeddings = torch.nn.Embedding(1024, model_bert.config.hidden_size)
# # Optional: Initialize the new embeddings using the original ones for the first 512 positions
# with torch.no_grad():
#     model_bert.embeddings.position_embeddings.weight[:512, :] = model_bert.embeddings.position_embeddings.weight[:512, :]


# model.resize_token_embeddings(new_vocab_size)

# and move our model over to the selected device
# model.to(device)












# def extend_position_embeddings(model, new_max_length):
#     old_embeddings = model.embeddings.position_embeddings.weight
#     old_max_length, embedding_size = old_embeddings.size()

#     if new_max_length <= old_max_length:
#         return model  # No extension needed if the new max length is less than or equal to the old max length

#     # Initialize new embeddings for extended positions
#     new_embeddings = model.embeddings.position_embeddings.__class__(
#         new_max_length, embedding_size
#     )
#     new_embeddings.weight.data[:old_max_length] = old_embeddings.data
#     new_embeddings.weight.data[old_max_length:] = model.embeddings.position_embeddings.weight.data[-1][None, :].repeat(new_max_length - old_max_length, 1)

#     model.embeddings.position_embeddings = new_embeddings
#     return model

from transformers import BertModel

# encoder = BertGenerationEncoder.from_pretrained("microsoft/MiniLM-L12-H384-uncased", bos_token_id=101, eos_token_id=102)
# new_max_length = 1024





config = BertConfig.from_pretrained("microsoft/MiniLM-L12-H384-uncased")
# Change max_position_embeddings to 1024 in the config
config.max_position_embeddings = 1024
# Load the model with the modified config

encoder = BertModel(config)
encoder.embeddings.position_embeddings = torch.nn.Embedding(1024, encoder.config.hidden_size)
with torch.no_grad():
    encoder.embeddings.position_embeddings.weight[:512, :] = encoder.embeddings.position_embeddings.weight[:512, :]



# encoder = extend_position_embeddings(encoder, new_max_length)
# encoder.config.max_position_embeddings = new_max_length


decoder = BertGenerationDecoder.from_pretrained(
    "microsoft/MiniLM-L12-H384-uncased", add_cross_attention=True, is_decoder=True, bos_token_id=101, eos_token_id=102
)



bert2bert = EncoderDecoderModel(encoder=encoder, decoder=decoder)


bert2bert.config.decoder_start_token_id = tokenizer.cls_token_id
bert2bert.config.eos_token_id = tokenizer.sep_token_id  # Typically the SEP token for BERT
bert2bert.config.pad_token_id = tokenizer.pad_token_id


input_ids = tokenizer(
    "This is the first sentence. This is the second sentence.", add_special_tokens=False, return_tensors="pt"
).input_ids

outputs = bert2bert.generate(input_ids)
print(outputs)
print(tokenizer.decode(outputs[0]))






# Create a TensorDataset and DataLoader
dataset = TensorDataset(source_encodings['input_ids'], source_encodings['attention_mask'], target_encodings['input_ids'])
loader = DataLoader(dataset, batch_size=2, shuffle=True)

# Training loop

bert2bert.to(device)
bert2bert.train()

from transformers import AdamW
optimizer = AdamW(bert2bert.parameters(), lr=5e-5)

for epoch in range(4):  # Number of epochs
    for batch in loader:
        optimizer.zero_grad()
        input_ids, attention_mask, labels = [x.to(device) for x in batch]
        print(input_ids.shape , attention_mask.shape , labels.shape )
        outputs = bert2bert(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch}, Loss: {loss.item()}")


You are using a model of type bert to instantiate a model of type bert-generation. This is not supported for all configurations of models and can yield errors.
Some weights of BertGenerationDecoder were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['encoder.layer.0.crossattention.output.LayerNorm.bias', 'encoder.layer.0.crossattention.output.LayerNorm.weight', 'encoder.layer.0.crossattention.output.dense.bias', 'encoder.layer.0.crossattention.output.dense.weight', 'encoder.layer.0.crossattention.self.key.bias', 'encoder.layer.0.crossattention.self.key.weight', 'encoder.layer.0.crossattention.self.query.bias', 'encoder.layer.0.crossattention.self.query.weight', 'encoder.layer.0.crossattention.self.value.bias', 'encoder.layer.0.crossattention.self.value.weight', 'encoder.layer.1.crossattention.output.LayerNorm.bias', 'encoder.layer.1.crossattention.output.LayerNorm.weight', 'encoder.layer.1.crossattention.output.dense.bias', 'e

tensor([[101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
         101, 101, 101, 101, 101, 101]])
NOP NOP NOP NOP NOP NOP NOP NOP NOP NOP NOP NOP NOP NOP NOP NOP NOP NOP NOP NOP
torch.Size([2, 980]) torch.Size([2, 980]) torch.Size([2, 26])


/home/raisul/anaconda3/envs/pytorch/lib/python3.12/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/raisul/anaconda3/envs/pytorch/lib/python3.12/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:629: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [87,0,0], thread: [0,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: 

RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling cublasLtMatmul with transpose_mat1 1 transpose_mat2 0 m 384 n 52 k 384 mat1_ld 384 mat2_ld 384 result_ld 384 abcType 0 computeType 68 scaleType 0

In [ ]:
from transformers import BertConfig, EncoderDecoderConfig, EncoderDecoderModel

# Initializing a BERT google-bert/bert-base-uncased style configuration
# config_encoder = BertConfig()







config = BertConfig.from_pretrained("microsoft/MiniLM-L12-H384-uncased")
# Change max_position_embeddings to 1024 in the config
config.max_position_embeddings = 1024
# Load the model with the modified config

model = BertModel(config)
# Access and modify the positional embeddings
# The model's `bert` attribute holds the BERT layers
model_bert = model.bert  # This is the BERT model itself (the backbone)
# Resize the position embeddings to accommodate the new max length (1024 tokens)
model_bert.embeddings.position_embeddings = torch.nn.Embedding(1024, model_bert.config.hidden_size)
# Optional: Initialize the new embeddings using the original ones for the first 512 positions
with torch.no_grad():
    model_bert.embeddings.position_embeddings.weight[:512, :] = model_bert.embeddings.position_embeddings.weight[:512, :]








config_decoder = BertConfig()

config = EncoderDecoderConfig.from_encoder_decoder_configs(config_encoder, config_decoder)

# Initializing a Bert2Bert model (with random weights) from the google-bert/bert-base-uncased style configurations
model = EncoderDecoderModel(config=config)

# Accessing the model configuration
config_encoder = model.config.encoder
config_decoder = model.config.decoder
# set decoder config to causal lm
config_decoder.is_decoder = True
config_decoder.add_cross_attention = True

# Saving the model, including its configuration
model.save_pretrained("my-model")

# loading model and config from pretrained folder
encoder_decoder_config = EncoderDecoderConfig.from_pretrained("my-model")
model = EncoderDecoderModel.from_pretrained("my-model", config=encoder_decoder_config)

In [ ]:

def extend_position_embeddings(model, new_max_length):
    # Extend position embeddings for both encoder and decoder
    for part in [model.encoder, model.decoder]:
        # Access the embeddings layer
        old_embeddings = part.embeddings.position_embeddings.weight
        old_max_length, embedding_size = old_embeddings.size()

        if new_max_length > old_max_length:
            # Create new embeddings and initialize
            new_embeddings = torch.nn.Embedding(new_max_length, embedding_size)
            new_embeddings.weight.data[:old_max_length] = old_embeddings.data
            new_embeddings.weight.data[old_max_length:] = old_embeddings.data[-1][None, :].repeat(new_max_length - old_max_length, 1)
            
            # Replace the old embeddings with the new ones
            part.embeddings.position_embeddings = new_embeddings

# Load a pre-trained EncoderDecoderModel
bert2bert = EncoderDecoderModel.from_encoder_decoder_pretrained("./../models/"+EXPERIMENT_NAME+"_model.ckpt", 'bert-base-uncased')

# # Extend position embeddings to 1024 tokens
# new_max_length = 1024
# extend_position_embeddings(model, new_max_length)

# # Optionally, update the configuration to reflect new maximum position embeddings
# model.config.encoder.max_position_embeddings = new_max_length
# model.config.decoder.max_position_embeddings = new_max_length

In [ ]:
bert2bert.config.decoder_start_token_id = tokenizer.cls_token_id
bert2bert.config.eos_token_id = tokenizer.sep_token_id
bert2bert.config.pad_token_id = tokenizer.pad_token_id
bert2bert.config.vocab_size = bert2bert.config.decoder.vocab_size

In [ ]:
# bert2bert.config.decoder_start_token_id = tokenizer.cls_token_id
# bert2bert.config.eos_token_id = tokenizer.sep_token_id
# bert2bert.config.pad_token_id = tokenizer.pad_token_id
# bert2bert.config.vocab_size = bert2bert.config.decoder.vocab_size

# bert2bert = EncoderDecoderModel.from_pretrained("google/roberta2roberta_L-24_discofuse")
# bert2bert.config.decoder.num_labels = 3
input_ids = tokenizer(
    "This is the first sentence. This is the second sentence.", add_special_tokens=False, return_tensors="pt"
).input_ids

outputs = bert2bert.generate(input_ids)
print(outputs)
print(tokenizer.decode(outputs[0]))

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Create a TensorDataset and DataLoader
dataset = TensorDataset(source_encodings['input_ids'], source_encodings['attention_mask'], target_encodings['input_ids'])
loader = DataLoader(dataset, batch_size=2, shuffle=True)

# Training loop

bert2bert.to(device)
bert2bert.train()

from transformers import AdamW
optimizer = AdamW(bert2bert.parameters(), lr=5e-5)

for epoch in range(4):  # Number of epochs
    for batch in loader:
        optimizer.zero_grad()
        input_ids, attention_mask, labels = [x.to(device) for x in batch]
        # print(input_ids.shape , attention_mask.shape , labels.shape )
        outputs = bert2bert(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch}, Loss: {loss.item()}")

In [ ]:

print(train_params[3])
print(text[-1])

In [ ]:
# text[51].split(delim)

We'll assign a 50% probability of using the genuine next sentence, and 50% probability of using another random sentence.

To make this simpler, we'll create a *'bag'* of individual sentences to pull from when selecting a random sentence B.

In [ ]:
delim = '\n'
bag = [instruction for instruction_cluster in text for instruction in instruction_cluster.split(delim)  if instruction!= '']
bag_size = len(bag)
print(bag_size , len(text))

And now we create our 50/50 NIP training data.

In [ ]:
import random

history = []
next_instruction = []
label = []


instruction_pages = []
for instruction_cluster in text:
    instructions = [
        instruction for instruction in instruction_cluster.split(delim) if instruction != ''
    ]

    instruction_pages.append(instructions)

        
print(len(instruction_pages))
print(instruction_pages[0])

for instruction_page in instruction_pages:
        # this is IsNextSentence
        history.append(delim.join(instruction_page))
        next_instruction.append(instruction_page[-1])
        label.append(0)


In [ ]:
inputs = tokenizer(history, next_instruction, return_tensors='pt', 
                   max_length=MAX_TOKEN_LEN, truncation=True, padding=True)
ground_truth = inputs.copy() 

In [ ]:
inputs.keys()

We can see that the *token_type_ids* tensors have been built correctly (eg **1** indicating sentence B tokens) by checking the first instance of *token_type_ids*:

And the labels tensor is simply a clone of the input_ids tensor before masking.

In [ ]:

inputs['next_sentence_label'] = torch.LongTensor([label]).T
inputs['labels'] = inputs.input_ids.detach().clone()

Now we mask tokens in the input_ids tensor using the 15% probability for MLM - ensuring we don't mask CLS, SEP, or PAD tokens.

In [ ]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [ ]:
mask_arr.shape
# inputs.input_ids.shape[0]

And now take the indices of each True value within each vector.

In [ ]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [ ]:
len (selection) , len(inputs.input_ids), selection[0:3]

Then apply these indices to each row in input_ids, assigning each value at these indices a value of 103.

In [ ]:
masked_labels = []
for i in range(inputs.input_ids.shape[0]):
    masked_labels.append(inputs.input_ids[i, selection[i]])
    inputs.input_ids[i, selection[i]] = 103
# masked_labels[0]
inputs["mask_arr"] = mask_arr

In [ ]:
inputs.keys()

The `inputs` tensors are now ready, and we can begin building the model input pipeline for training. We first create a PyTorch dataset from our data.

In [ ]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

Initialize our data using the `MeditationDataset` class.

In [ ]:
dataset = MeditationsDataset(inputs)
# print(inputs.shape)

In [ ]:


train_dataset  = torch.utils.data.Subset(dataset, range(len(train_text)))
validation_dataset = torch.utils.data.Subset(dataset, range(len(train_text) , len(dataset)))

len(train_dataset) , len(validation_dataset)

And initialize the dataloader, which we'll be using to load our data into the model during training.

In [ ]:

train_loader      = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False )



Activate the training mode of our model, and initialize our optimizer (Adam with weighted decay - reduces chance of overfitting).

In [ ]:
from sklearn.metrics import precision_recall_fscore_support , accuracy_score
import numpy as np


Now we can move onto the training loop, we'll train for a couple of epochs (change `epochs` to modify this).

In [ ]:
# odict_keys(['loss', 'prediction_logits', 'seq_relationship_logits'])

In [ ]:

from numpy import *
import math
import matplotlib.pyplot as plt


def plot_graph(training_data, validation_data , label ):

    font_size = 10
    x_labels = [ i for i in range(len(training_data)) ]

    plt.ylabel(' F1 ',fontsize=font_size)
    plt.plot(x_labels, training_data , 'r') 
    plt.plot(x_labels, validation_data , 'b') 
    plt.xlabel("Epoch", fontsize=font_size)
    plt.title(label,fontsize=font_size)
    plt.legend(['Training', 'Validation'], loc='upper left') 
    
    plt.savefig('./../../results/'+EXPERIMENT_NAME+label+'.pdf')
    plt.show()
    with open('./../../results/'+EXPERIMENT_NAME+label+'.json', 'w') as json_file:
        json.dump([training_data, validation_data , label], json_file)



In [ ]:
from transformers import AdamW
from tqdm import tqdm  # for our progress bar
from itertools import chain



# initialize optimizer
optim = AdamW(model.parameters(), lr=3e-4 ,weight_decay=0.0001)

# Number of training steps per epoch
train_steps = len(train_loader)

# Define the number of total training steps and warmup steps
total_steps = train_steps * epochs
warmup_steps = int(total_steps * 0.1)  # 10% of total steps as warmup

# Scheduler setup
scheduler = get_linear_schedule_with_warmup(optim,
                                            num_warmup_steps=warmup_steps,
                                            num_training_steps=total_steps)

In [ ]:


counter = 0

global_instruction_metrices = []
global_masked_token_metrices = []

v_global_instruction_metrices = []
v_global_masked_token_metrices = []


for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    train_loop = tqdm(train_loader, leave=True)
    
    
    instruction_predictions_all, instruction_ground_truths_all = None, None
    masked_token_predictions_all, masked_token_ground_truths_all = None, None
    seq_predictions_all, seq_ground_truths_all = None, None
    
    # activate training mode
    model.train()
    for N,batch in enumerate(train_loop):

        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        next_sentence_label = batch['next_sentence_label'].to(device)
        batch_mask_arr = batch ['mask_arr']
        labels = batch['labels'].to(device)

        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        next_sentence_label=next_sentence_label,
                        labels=labels)


        token_prediction = torch.argmax(outputs.prediction_logits, axis=-1)
       


        # batch_masks = selection [BATCH_SIZE*N : (BATCH_SIZE*(N+1))]
        # print('batch_masks old: ',batch_masks)

        # print(batch ['mask_arr'].shape) #torch.flatten(mask_arr[i].nonzero()).tolist()
        batch_masks =   [ torch.flatten(bm.nonzero()).tolist()  for bm in batch_mask_arr]    # torch.flatten(batch ['mask_arr'].nonzero()).tolist()
        # print('batch_masks new: ',batch_masks)
        
        # print("BATCH_SIZE*N : (BATCH_SIZE*(N+1)): ",BATCH_SIZE*N , (BATCH_SIZE*(N+1)) )
        # print("batch_masks:",batch_masks)



        

        masked_token_prediction = [ token[batch_masks[t]].tolist() for t,token in enumerate(token_prediction) ]
        masked_token_prediction = list(chain.from_iterable(masked_token_prediction))
        
        masked_token_ground_truth   = [ token[batch_masks[t]].tolist() for t,token in enumerate(labels) ]
        masked_token_ground_truth = list(chain.from_iterable(masked_token_ground_truth))
        

        # print(token_prediction , token_ground_truth)

        # token_prediction = token_prediction.detach().cpu().numpy().flatten()
        # token_ground_truth = labels.detach().cpu().numpy().flatten()

        # print("token_prediction  : ", token_prediction)
        # print("token_ground_truth: ", token_ground_truth)


        seq_predictions   = token_prediction.detach().cpu().numpy().flatten()
        seq_ground_truths = labels.detach().cpu().numpy().flatten()

        
        instruction_prediction = torch.argmax(outputs.seq_relationship_logits, axis=-1)
        instruction_prediction   = instruction_prediction.detach().cpu().numpy().flatten()
        instruction_ground_truth = next_sentence_label.detach().cpu().numpy().flatten()
        
        if N==0:
            instruction_predictions_all   = instruction_prediction
            instruction_ground_truths_all = instruction_ground_truth
            
            masked_token_predictions_all         = masked_token_prediction
            masked_token_ground_truths_all       = masked_token_ground_truth  


            seq_predictions_all = seq_predictions
            seq_ground_truths_all = seq_ground_truths
            
        else:
            instruction_predictions_all   = np.concatenate((instruction_predictions_all, instruction_prediction))
            instruction_ground_truths_all = np.concatenate((instruction_ground_truths_all, instruction_ground_truth))
            
            masked_token_predictions_all   = np.concatenate((masked_token_predictions_all, masked_token_prediction))
            masked_token_ground_truths_all = np.concatenate((masked_token_ground_truths_all, masked_token_ground_truth))

            seq_predictions_all = np.concatenate((seq_predictions_all, seq_predictions))
            seq_ground_truths_all = np.concatenate((seq_ground_truths_all, seq_ground_truths))
            

        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        scheduler.step()
        # print relevant info to progress bar
        train_loop.set_description(f'Epoch {epoch}')
        train_loop.set_postfix(loss=loss.item())
    
    
    instruction_accuracy = (accuracy_score(instruction_ground_truths_all,instruction_predictions_all))
    instruction_precision, instruction_recall, instruction_f1, _ = precision_recall_fscore_support(instruction_ground_truths_all,instruction_predictions_all, average='binary')
    
    masked_token_accuracy = (accuracy_score(masked_token_ground_truths_all, masked_token_predictions_all))
    masked_token_precision, masked_token_recall, masked_token_f1, _ = precision_recall_fscore_support(masked_token_ground_truths_all,masked_token_predictions_all,average='weighted')

    seq_precision, seq_recall, seq_f1, _ = precision_recall_fscore_support(seq_ground_truths_all,seq_predictions_all,average='weighted')
    
    print("Training: ",  ' Instruction f1: ', instruction_f1 , '  Masked Token f1: ',masked_token_f1 , ' masked_token_precision: ', masked_token_precision, ' masked_token_recall: ', masked_token_recall, ' masked_token_accuracy ', masked_token_accuracy,"    SEQ F1",seq_f1)
    global_instruction_metrices.append(instruction_f1)
    global_masked_token_metrices.append( masked_token_f1) 

    ###########################################
    ###############  EVAL Validation  #########
    ###########################################
    with torch.no_grad():
        model.eval()
#         v_predictions_all, v_ground_truths_all = None, None
        
        v_instruction_predictions_all, v_instruction_ground_truths_all = None, None
        v_masked_token_predictions_all, v_masked_token_ground_truths_all = None, None
        v_seq_predictions_all, v_seq_ground_truths_all = None, None
    
    
        validation_loop = tqdm(validation_loader, leave=True)
        for N,v_batch in enumerate(validation_loop):
            
            
            
            v_input_ids = v_batch['input_ids'].to(device)
            v_token_type_ids = v_batch['token_type_ids'].to(device)
            v_attention_mask = v_batch['attention_mask'].to(device)
            v_next_sentence_label = v_batch['next_sentence_label'].to(device)
            v_mask_arr = v_batch ['mask_arr']
            v_labels = v_batch['labels'].to(device)
            # process
            v_outputs = model(v_input_ids, attention_mask=v_attention_mask,
                            token_type_ids=v_token_type_ids,
                            next_sentence_label=v_next_sentence_label,
                            labels=v_labels)
        
            v_token_prediction = torch.argmax(v_outputs.prediction_logits, axis=-1)

                    

            v_batch_masks =   [ torch.flatten(bm.nonzero()).tolist()  for bm in v_mask_arr]
            
            v_masked_token_prediction = [ token[v_batch_masks[t]].tolist() for t,token in enumerate(v_token_prediction) ]
            v_masked_token_prediction = list(chain.from_iterable(v_masked_token_prediction))
            
            v_masked_token_ground_truth   = [ token[v_batch_masks[t]].tolist() for t,token in enumerate(v_labels) ]
            v_masked_token_ground_truth = list(chain.from_iterable(v_masked_token_ground_truth))
    
            
            

            v_seq_prediction = v_token_prediction.detach().cpu().numpy().flatten()
            v_seq_ground_truth = v_labels.detach().cpu().numpy().flatten()


            

            # token_prediction = [ token[batch_masks[t]].tolist() for t,token in enumerate(token_prediction) ]
            # token_prediction = list(chain.from_iterable(token_prediction))
            
            # token_ground_truth   = [ token[batch_masks[t]].tolist() for t,token in enumerate(labels) ]
            # token_ground_truth = list(chain.from_iterable(token_ground_truth))

            
            v_instruction_prediction = torch.argmax(v_outputs.seq_relationship_logits, axis=-1)
            v_instruction_prediction   = v_instruction_prediction.detach().cpu().numpy().flatten()
            v_instruction_ground_truth = v_next_sentence_label.detach().cpu().numpy().flatten()



        

            if N==0:
                v_instruction_predictions_all   = v_instruction_prediction
                v_instruction_ground_truths_all = v_instruction_ground_truth

                v_masked_token_predictions_all   = v_masked_token_prediction
                v_masked_token_ground_truths_all = v_masked_token_ground_truth
                
                v_seq_predictions_all= v_seq_prediction
                v_seq_ground_truths_all = v_seq_ground_truth

        

            else:
                v_instruction_predictions_all   = np.concatenate((v_instruction_predictions_all, v_instruction_prediction))
                v_instruction_ground_truths_all = np.concatenate((v_instruction_ground_truths_all, v_instruction_ground_truth))

                v_masked_token_predictions_all   = np.concatenate((v_masked_token_predictions_all, v_masked_token_prediction ))
                v_masked_token_ground_truths_all = np.concatenate((v_masked_token_ground_truths_all, v_masked_token_ground_truth ))
                
                v_seq_predictions_all =np.concatenate((v_seq_predictions_all, v_seq_prediction ))
                v_seq_ground_truths_all =np.concatenate((v_seq_ground_truths_all, v_seq_ground_truth ))
                

            

        v_instruction_accuracy = (accuracy_score(v_instruction_ground_truths_all,v_instruction_predictions_all))
        v_instruction_precision, v_instruction_recall, v_instruction_f1, _ = precision_recall_fscore_support(v_instruction_ground_truths_all,v_instruction_predictions_all, average='binary')


        v_masked_token_accuracy = (accuracy_score(v_masked_token_ground_truths_all, v_masked_token_predictions_all))
        v_masked_token_precision, v_masked_token_recall, v_masked_token_f1, _ = precision_recall_fscore_support(v_masked_token_ground_truths_all,v_masked_token_predictions_all,average='weighted')


        v_seq_accuracy = (accuracy_score(v_seq_predictions_all, v_seq_ground_truths_all))
        v_seq_precision, v_seq_recall, v_seq_f1, _ = precision_recall_fscore_support(v_seq_ground_truths_all,v_seq_predictions_all,average='weighted')
    
        print(epoch,"Validation: ", "Instruction F1: ", v_instruction_f1,  "   v_masked_token_ F1: ",v_masked_token_f1,'v_seq_accuracy: ',v_seq_accuracy,' v_masked_token_precision : ',v_masked_token_precision,'v_masked_token_recall:', v_masked_token_recall," V SEQ F1: ", v_seq_f1)
        
        v_global_instruction_metrices.append(v_instruction_f1)
        v_global_masked_token_metrices.append(v_masked_token_f1) 

    
    plot_graph(global_instruction_metrices, v_global_instruction_metrices, 'Next Sentence Prediction Scores')
    plot_graph(global_masked_token_metrices, v_global_masked_token_metrices, 'Masked Token Prediction Scores')
    model.save_pretrained("./../../models/"+EXPERIMENT_NAME+"_model.ckpt")

In [ ]:
import torch
from torch import nn
from transformers import BertModel, BertConfig, BertTokenizer

class BertSummarizer(nn.Module):
    def __init__(self, encoder_name, config_decoder):
        super().__init__()
        self.encoder = BertModel.from_pretrained(encoder_name)
        self.decoder = nn.TransformerDecoder(
            nn.TransformerDecoderLayer(d_model=config_decoder.hidden_size, nhead=config_decoder.num_attention_heads),
            num_layers=config_decoder.num_hidden_layers
        )
        self.output_layer = nn.Linear(config_decoder.hidden_size, config_decoder.vocab_size)

    def forward(self, input_ids, decoder_input_ids, attention_mask=None, tgt_mask=None):
        encoder_outputs = self.encoder(input_ids, attention_mask=attention_mask)
        encoder_hidden_states = encoder_outputs.last_hidden_state
        decoder_outputs = self.decoder(tgt=decoder_input_ids,
                                       memory=encoder_hidden_states,
                                       tgt_mask=tgt_mask)
        outputs = self.output_layer(decoder_outputs)
        return outputs

# Configuration for the decoder
decoder_config = BertConfig(
    hidden_size=768,  # Match BERT's hidden size
    vocab_size=30522,  # Match BERT's vocab size
    num_attention_heads=12,
    num_hidden_layers=6
)

# Instantiate the summarizer
summarizer = BertSummarizer("bert-base-uncased", decoder_config)
